# Ninety One Technical Assessment – Portfolio Risk Analysis  
### *Prepared by Nicholas de Clercq*

This notebook explores the characteristics, risk dynamics, and ESG profile of an equity portfolio provided for analysis.  
All analytics and visualisations are performed in **Python**, with results designed to demonstrate both technical proficiency and interpretive insight.

---

## Project Overview

The analysis is structured in four parts:

1. **Current Portfolio Characteristics**  
   - Composition by sector, country, and position size  
   - Concentration and diversification metrics  
   - Activeness versus the benchmark  

2. **Return Analysis**  
   - Returns  
   - Active returns
   - Fund size through time
   - Return attribution  
     - Top 10 share contributors and detractors  
     - Sector return attribution  
     - Region return attribution  
   - Sector exposures through time  
   - Regional exposures through time  

3. **Risk Analysis**  
   - Total risk and active risk trends  
   - Factor exposures and beta evolution  
   - Contribution to risk by sector, region, and asset  
   - Portfolio diversification and correlation insights  
   - drawdowns, sharpe ratio, rolling standard deviation, VaR, cVaR ...

4. **ESG Analysis**  
   - Weighted ESG scores and trends  
   - Breakdown of Environmental, Social, and Governance dimensions  
   - Relationship between ESG quality and risk contribution  

---

### Objective

The goal is to identify and clearly articulate the **key drivers of portfolio risk, performance, and ESG trends** —  
and to demonstrate how these factors have evolved over time.

---

*All charts are interactive and were generated using `plotly.express` with a custom Ninety One colour palette.*


In [638]:
import importlib, portfolio_utils as pu
importlib.reload(pu)
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.renderers.default = "notebook_connected"

# Optional: sizing/style
px.defaults.width = 720
px.defaults.height = 420
px.defaults.template = "plotly_white"
# jupyter nbconvert --to html --no-input "portfolio_analysis_jupyter.ipynb"
# jupyter nbconvert --execute --to html --no-input "portfolio_analysis_jupyter.ipynb"


from portfolio_utils import *

### 1. Current Portfolio Characteristics

In the following sectin we can take a look at the portfolio's current characteristics. Starting with the current holdings of the portfolios.

In [639]:
df = load_data("Risk Analyst Case study_092025.csv")
df = add_derived_columns(df)
pa = PortfolioAnalyzer(df)


In [640]:
top10_pie_latest(df, by="Weight (%)").show()

In [641]:
sector_pie_latest(df).show()

In [642]:
country_pie_latest(df).show()

This portfolio shows a large exposure to emerginf market countires such as China and India as well as exposures to South American countires and South Africa, typical of a fund that would be benchmarked to the the MSIC emerging markets index.

To Further understand the current drivers of risk in the portfolio we can take a look at the most recent contributors to the overall risk of the portfolio.

In [643]:
risk_contrib_bar(df).show()

In [644]:
sector_risk_contrib(df).show()

In [645]:
country_risk_contrib(df).show()

The majority of the risk comes unsuprisingly from its largest holdings.

### 2. Portfolio Return Analysis

To quantify realised performance, we compute **periodic portfolio returns** from asset price (or index) levels.  
Let $P_{i,t}$ be the price (or total-return index level) of asset $i$ at time $t$, and $w_{i,t-1}$ the weight held at the **start** of period $t$.

**Step 1 — Asset-level returns**  
Simple returns:  
$$
r_{i,t}=\frac{P_{i,t}}{P_{i,t-1}}-1
$$
Log returns (optional):  
$$
r_{i,t}^{(\log)}=\ln\!\left(\frac{P_{i,t}}{P_{i,t-1}}\right)
$$

**Step 2 — Portfolio-level return (pre-costs)**  
Using beginning-of-period weights:  
$$
R_{p,t}=\sum_{i=1}^{N} w_{i,t-1}\,r_{i,t}
$$

**Step 3 — Compounding over a horizon $T$**  
Cumulative simple return:  
$$
R_{p,1:T}=\prod_{t=1}^{T}\bigl(1+R_{p,t}\bigr)-1
$$
Cumulative log return (if using logs):  
$$
R_{p,1:T}^{(\log)}=\sum_{t=1}^{T} R_{p,t}^{(\log)}, \quad \text{where } R_{p,t}^{(\log)}=\ln\!\bigl(1+R_{p,t}\bigr)
$$

**Step 4 — Rebalancing through time**  
If the portfolio is rebalanced (e.g., monthly), compute $R_{p,t}$ each period using the updated weights $w_{i,t-1}$.  
Weights should satisfy $\sum_i w_{i,t-1}=1$ (or include cash if not fully invested).

---



In [646]:
plot_port_vs_bench(df)

In [647]:
fig, tri_df = portfolio_vs_benchmarks(df)
fig.show()

In [648]:
fig = monthly_bar_px_all(df)
fig.show()

The portfolio is clearly an actively managed fund with active weights against its benchmark. Over the period of 29 Sep 2022 to 29 Mar 2023, the fund has managed to outperform its benchmark as well as the MSCI world, the MSCI emerging market index and the JSE All Share index. Showing impressive skill from the fund manager.

### Retun Attribution

To gain a better understanding of where the outperformance has come from, we will perform a return attribution analysis on the portfolio. First we plot the expsoure of the fund to sectors, countires and shares through time.

In [649]:
sector_exposure_area(df).show()
country_exposure_area(df).show()

The sector and country exposure are fairly stable through time. To gain better insight into how these have contributed to porfolio performance we look at the following:

In [650]:
sector_return_contrib_area(df).show()
country_return_contrib_area(df).show()

In [651]:
sector_return_contrib_totals_bar(df).show()
country_return_contrib_totals_bar(df).show()

It is clear that the information technology sector made the majority of the gains with a meaningful contribution from the communication services sector. China and Taiwan delivered the majority of the positive performance with Brazil the only large negative contributing region.

We can futher this analysis by looking into which share were the top 10 contributors and detractors to performance over the full period.

In [652]:
top_share_contributors_bar(df).show()
top_share_detractors_bar(df).show()

Taiwan Semiconductor manufact co was the notable performer over the period with a return of 10.5%. Tencent and Alibaba making meaningful contributions as well. On Hapvida and Meituan were the most negative dectractors to performance.

### 3. Risk Analysis

In this section we will explore the overall portfolio risk and how that has changed through time.